In [1]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology", "Others"]
# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

def get_report(base_path, test_name, extract_func, mlb, classes, verbose = False):
    df = pd.read_csv(base_path+test_name+"/test_generic_test_0.csv")
    if verbose:
        print("------------------")
        print("Prompt:")
        print(df.prompt[0])
        print("------------------")
        print()
        print()
        for i in range(0,3):
            print("------------------")
            print("Pre-Extraction:")
            print("------------------")
            print("Normalized Tweet: ", df.iloc[i].normalized_tweet)
            print("Response: ", df.iloc[i]['response'])
        print()
    df['annotations'] = df['annotations'].apply(lambda x: extract_func(x, classes))
    df['response'] = df['response'].apply(lambda x: extract_func(x, classes))
    if verbose:
        for i in range(0,3):
            print("------------------")
            print("Post-Extraction:")
            print("------------------")
            print("Normalized Tweet: ", df.iloc[i].normalized_tweet)
            print("Response: ", df.iloc[i]['test'+test_name])
        print()
    y_true = mlb.fit_transform(df['annotations'])
    y_pred = mlb.transform(df['response'])
    report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)
    return pd.DataFrame(report).transpose()

# No Fine Tuning

In [2]:
base_path = "../data/vicuna_4bit/"

report_multilabel_no_fine_tune_v01 = get_report(base_path, "multi_label_no_fine_tune_v01", llm_utils.extract_multilabel_list, mlb, classes)
report_multilabel_no_fine_tune_v01_only_first_label_extracted = get_report(base_path, "multi_label_no_fine_tune_v01", llm_utils.extract_multilabel_list_only_first_class, mlb, classes)
report_multilabel_no_fine_tune_explanation_first_v01 = get_report(base_path, "multi_label_no_fine_tune_explanation_first_v01", llm_utils.extract_multilabel_list_explanation_first, mlb, classes)
report_multilabel_no_fine_tune_explanation_first_v01_only_first_label_extracted = get_report(base_path, "multi_label_no_fine_tune_explanation_first_v01", llm_utils.extract_multilabel_list_explanation_first_only_first_class, mlb, classes)

print()
print("----------------")
print("Multilabel v01 NO FINETUNING")
print("----------------")
print(report_multilabel_no_fine_tune_v01)
print("----------------")
print()
print("----------------")
print("Multilabel v01 NO FINETUNING Only First Label Extracted")
print("----------------")
print(report_multilabel_no_fine_tune_v01_only_first_label_extracted)
print("----------------")
print()
print("----------------")
print("Multilabel v01 NO FINETUNING Explanation First")
print("----------------")
print(report_multilabel_no_fine_tune_explanation_first_v01)
print("----------------")
print()
print("----------------")
print("Multilabel v01 NO FINETUNING Explanation First Only First Label Extracted")
print("----------------")
print(report_multilabel_no_fine_tune_explanation_first_v01_only_first_label_extracted)
print("----------------")


----------------
Multilabel v01 NO FINETUNING
----------------
                                    precision    recall  f1-score  support
War/Terror                           0.301956  0.968627  0.460391    255.0
Conspiracy Theory                    0.052098  0.800000  0.097826     45.0
Education                            0.019753  0.615385  0.038278     13.0
Election Campaign                    0.061728  0.757576  0.114155     33.0
Environment                          0.027228  0.785714  0.052632     14.0
Government/Public                    0.345339  0.560137  0.427261    291.0
Health                               0.082547  0.760870  0.148936     46.0
Immigration/Integration              0.043478  0.500000  0.080000     36.0
Justice/Crime                        0.226190  0.693431  0.341113    137.0
Labor/Employment                     0.042607  0.607143  0.079625     28.0
Macroeconomics/Economic Regulation   0.058228  0.370968  0.100656     62.0
Media/Journalism                    

/home/bruno/anaconda3/envs/p5/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['"Abortion & Reproductive Health Care"\n### Human:', '"Accident"', '"Accident",\n### Human:', '"Africa"', '"African Politicians and their Elections"\n\nExplanation: The topic of the tweet is about President Cyril Ramaphosa congratulating the re-elected Nigerian president on his election victory. This can be classified under two broader topics - "Foreign Government Officials Congratulating Other Foreign Governments Electoral Victories" and "African Politicians and their Elections". These are both relevant to the content of the tweet and provide a clear context for why these topics may be discussed in relation to it.\n### Human:', '"Afronaut"', '"Animals/Animal Rights"\n### Human:', '"Anti-corruption commissioner"', '"Assassination",\n\nExplanation: The tweet mentions the assassination of three individuals who are part of a reconciliation committee in southern S

# Multilabel Fine-Tuning

In [10]:
vicuna_lora_multilabel_with_rules_v02["binary_classification_reports"]

,label,f1_score_macro,precision_macro,recall_macro,support_macro,f1_score_class_0,support_class_0,f1_score_class_1,support_class_1,precision_class_0,recall_class_0,precision_class_1,recall_class_1
0,War/Terror,0.904691,0.946875,0.876365,1000,0.956634,745,0.852747,255,0.923750,0.991946,0.970000,0.760784
1,Conspiracy Theory,0.748212,0.716745,0.793310,1000,0.973059,955,0.523364,45,0.981876,0.964398,0.451613,0.622222
2,Education,0.760637,0.712243,0.840075,1000,0.991862,987,0.529412,13,0.995914,0.987842,0.428571,0.692308
3,Election Campaign,0.879725,0.920787,0.846417,1000,0.992784,967,0.766667,33,0.989723,0.995863,0.851852,0.696970
4,Environment,0.793509,0.804653,0.783179,1000,0.994425,986,0.592593,14,0.993921,0.994929,0.615385,0.571429
5,Government/Public,0.819079,0.824496,0.814280,1000,0.897418,709,0.740741,291,0.888122,0.906911,0.760870,0.721649
6,Health,0.789509,0.880511,0.735986,1000,0.984424,954,0.594595,46,0.975309,0.993711,0.785714,0.478261
7,Immigration/Integration,0.766301,0.884033,0.706259,1000,0.987147,964,0.545455,36,0.978593,0.995851,0.789474,0.416667
8,Justice/Crime,0.871180,0.919347,0.836113,1000,0.968697,863,0.773663,137,0.951902,0.986095,0.886792,0.686131
9,Labor/Employment,0.728367,0.752681,0.708627,1000,0.986147,972,0.470588,28,0.983623,0.988683,0.521739,0.428571


In [9]:
vicuna_lora_multilabel_with_rules_v02_df = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02/test_generic_test_0.csv")
vicuna_lora_multilabel_with_rules_v02_predictions_per_class, confusion_matrices, binary_classification_reports, multilabel_classification_reports = llm_utils.calculate_metrics_from_multilabel_list(vicuna_lora_multilabel_with_rules_v02_df, classes, llm_utils.extract_multilabel_list)
vicuna_lora_multilabel_with_rules_v02 = {"confusion_matrices": confusion_matrices, "binary_classification_reports": binary_classification_reports, "multilabel_classification_reports": multilabel_classification_reports}
vicuna_lora_multilabel_with_rules_v02["multilabel_classification_reports"]

,precision,recall,f1-score,support
War/Terror,0.970000,0.760784,0.852747,255.0
Conspiracy Theory,0.451613,0.622222,0.523364,45.0
Education,0.428571,0.692308,0.529412,13.0
Election Campaign,0.851852,0.696970,0.766667,33.0
Environment,0.615385,0.571429,0.592593,14.0
Government/Public,0.760870,0.721649,0.740741,291.0
Health,0.785714,0.478261,0.594595,46.0
Immigration/Integration,0.789474,0.416667,0.545455,36.0
Justice/Crime,0.886792,0.686131,0.773663,137.0
Labor/Employment,0.521739,0.428571,0.470588,28.0


In [5]:
base_path = "../data/vicuna_4bit/lora/"

In [6]:
report_multilabel_v01_128_rank_retest = get_report(base_path, "multilabel_without_context_v01_retest", llm_utils.extract_multilabel_list, mlb, classes)
#report_multilabel_v01 = get_report(base_path, "multilabel_without_context_v01", llm_utils.extract_multilabel_list, mlb, classes)
#report_multilabel_v01_256_rank = get_report(base_path, "multilabel_without_context_v01_256_rank", llm_utils.extract_multilabel_list, mlb, classes)
#report_multilabel_v02 = get_report(base_path, "multilabel_with_rules_v02", llm_utils.extract_multilabel_list, mlb, classes)
#report_multilabel_v02_256_rank = get_report(base_path, "multilabel_with_rules_v02_256_rank", llm_utils.extract_multilabel_list, mlb, classes)

print("All labels extracted from LLM response")
print()
print("----------------")
print("Multilabel v01 128 LoRA rank retest")
print("----------------")
print(report_multilabel_v01_128_rank_retest)
print("----------------")
print()
print("----------------")
print("Multilabel v01")
print("----------------")
print(report_multilabel_v01)
print("----------------")
print()
print("----------------")
print("Multilabel v01 256 LoRA rank")
print("----------------")
print(report_multilabel_v01_256_rank)
print("----------------")
print()
print("----------------")
print("Multilabel v02")
print("----------------")
print(report_multilabel_v02)
print("----------------")
print()
print("----------------")
print("Multilabel v02 256 LoRA rank")
print("----------------")
print(report_multilabel_v02_256_rank)
print("----------------")

Error for War/Terror: 'macro avg'
Error for Conspiracy Theory: 'macro avg'
Error for Education: 'macro avg'
Error for Election Campaign: 'macro avg'
Error for Environment: 'macro avg'
Error for Government/Public: 'macro avg'
Error for Health: 'macro avg'
Error for Immigration/Integration: 'macro avg'
Error for Justice/Crime: 'macro avg'
Error for Labor/Employment: 'macro avg'
Error for Macroeconomics/Economic Regulation: 'macro avg'
Error for Media/Journalism: 'macro avg'
Error for Religion: 'macro avg'
Error for Science/Technology: 'macro avg'
Error for Others: 'macro avg'
Error for micro avg: 'macro avg'
Error for macro avg: 'macro avg'
Error for weighted avg: 'macro avg'
Error for samples avg: 'macro avg'
                                 label f1_score_macro f1_score_class_0 support_class_0 f1_score_class_1 support_class_1
0                           War/Terror           None             None            None             None            None
1                    Conspiracy Theory    

NameError: name 'report_multilabel_v01' is not defined

# Metrics for multilabel but only the first class predicted is used

In [6]:
report_multilabel_v01_128_rank_retest = get_report(base_path, "multilabel_without_context_v01_retest", llm_utils.extract_multilabel_list_only_first_class, 
                                                   mlb, classes)

report_multilabel_v01 = get_report(base_path, "multilabel_without_context_v01", llm_utils.extract_multilabel_list_only_first_class,
                                   mlb, classes)

report_multilabel_v01_256_rank = get_report(base_path, "multilabel_without_context_v01_256_rank", llm_utils.extract_multilabel_list_only_first_class,
                                   mlb, classes)

report_multilabel_v02 = get_report(base_path, "multilabel_with_rules_v02", llm_utils.extract_multilabel_list_only_first_class,
                                   mlb, classes)

report_multilabel_v02_256_rank = get_report(base_path, "multilabel_with_rules_v02_256_rank", llm_utils.extract_multilabel_list_only_first_class,
                                   mlb, classes)

print("Only first label is extracted from LLM response")
print()
print("----------------")
print("Multilabel v01 retest")
print("----------------")
print(report_multilabel_v01)
print("----------------")
print()
print("----------------")
print("Multilabel v01")
print("----------------")
print(report_multilabel_v01)
print("----------------")
print()
print("----------------")
print("Multilabel v01 256 LoRA rank")
print("----------------")
print(report_multilabel_v01_256_rank)
print("----------------")
print()
print("----------------")
print("Multilabel v02")
print("----------------")
print(report_multilabel_v02)
print("----------------")
print()
print("----------------")
print("Multilabel v02 256 LoRA rank")
print("----------------")
print(report_multilabel_v02_256_rank)
print("----------------")

Only first label is extracted from LLM response

----------------
Multilabel v01 retest
----------------
                                    precision    recall  f1-score  support
War/Terror                           0.000000  0.000000  0.000000    255.0
Conspiracy Theory                    0.666667  0.222222  0.333333     45.0
Education                            1.000000  0.076923  0.142857     13.0
Election Campaign                    0.000000  0.000000  0.000000     33.0
Environment                          0.000000  0.000000  0.000000     14.0
Government/Public                    1.000000  0.010309  0.020408    291.0
Health                               0.000000  0.000000  0.000000     46.0
Immigration/Integration              0.000000  0.000000  0.000000     36.0
Justice/Crime                        0.000000  0.000000  0.000000    137.0
Labor/Employment                     0.000000  0.000000  0.000000     28.0
Macroeconomics/Economic Regulation   0.000000  0.000000  0.000000     

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/anaconda3/envs/my_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['2017. It features an image of one of the performers at the event.\n\n### Response:\nMusic\n\n### Outro:\n\n### Response:\nOthers\n\n### Summary:\nThe tweet presents a quote by Nelson Mandela : " Education is the most powerful weapon you can use to change the', 'Government/Public\n\n### Analysis:\n\n### Response:\nOthers\n\n### Outro:\n\n### Human:', 'Government/Public\n\n### Assistant:', 'Government/Public\n\n### Human:', 'Health\n\n### Analysis:\nThe tweet is related to health issues and measures taken by the authorities to prevent spreading of coronavirus within CAR borders. It does not contain any information on war or terror', 'Health\n\n### Assistant:', 'Health\n\n### Human:', 'Immigration/Integration\n\n### Assistant:', 'Immigration/Integration\n\n### Human:', 'Infrastructure\n\n### Assistant:', 'Justice/C